In [1]:
import bs4
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [2]:
embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')
vectorstore = Chroma(embedding_function=embeddings, persist_directory='./vectorstore/chroma_db')
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

prompt = PromptTemplate.from_template("""
Você é um assistente para tarefas de resposta a perguntas. 
Use as seguintes partes do contexto recuperado para responder à pergunta. 
Se você não sabe a resposta, basta dizer que não sabe. 
Use no máximo três frases e mantenha a resposta concisa.
Responda em português.

Questão: {question} 

Contexto: {context} 

Resposta:
""")

llm = Ollama(model='mistral:7b')


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("O que são containers?")

' Containers são unidades de software que isolam aplicativos e seus recursos, funcionando dentro de uma infraestrutura de Linux, sem serem máquinas virtuais. Elas podem compartilhar o mesmo kernel do sistema operacional e usar recursos de armazenamento e rede de forma unida, reduzindo o tamanho e a complexidade das imagens.'